# This is now testing the best model out of the Spacy Fine tune

## Setting up the local model

In [1]:
import spacy

model_path = 'newmodel/model-best'

# Load the custom spaCy model
nlp = spacy.load(model_path)


## Generating random data

In [2]:
import pandas as pd
import random
from faker import Faker
faker = Faker()

# Initialize empty lists to store mock data and labels
mock = []
mock_labels = []
i = 0
# Generate and store 300 mock invoices
while i < 1000:
    name = faker.name()
    street = faker.street_address()
    city = faker.city()
    state = faker.state()
    zip = faker.zipcode()
    word1 = faker.word()
    choice1 = random.choice(["Laptop", "Smartphone", "Tablet", "Monitor", "Keyboard"])
    make1= random.choice(["X1", "S3", "T5", "M2", "K4"])
    word2 = faker.word()
    choice2 = random.choice(["Headphones", "Mouse", "Printer", "Scanner", "Webcam"])
    make2=random.choice(["H6", "M3", "P2", "S4", "W1"])
    invnos = random.randint(100, 999)
    invmonth =  random.randint(1, 12)
    invday = random.randint(1, 28)
    duemonth =  random.randint(1, 12)
    dueday = random.randint(1, 28)
    balance = random.uniform(100, 999999999)
    year = 2023
    invoice_text = f"""
    Dear Customer,

    Invoice Number: INV-2023-{invnos}
    Invoice Date: 2023-{invmonth}-{invday}
    Due Date: 2023-{duemonth}-{dueday}

    Billing Information:
    Recipient Name: {name}
    Recipient Address: {street}, {city}, {state} {zip}

    Product Details:
    - Description: {word1} {choice1} {make1}
      Quantity: {random.randint(1, 5)}
      Unit Price: ${random.uniform(50, 1000):.2f}
    - Description: {word2} {choice2} {make2}
      Quantity: {random.randint(1, 5)}
      Unit Price: ${random.uniform(10, 200):.2f}

    Balance Due: ${balance}

    Thank you for your business!
    """

    # Append the generated invoice text to the mock list
    mock.append([i+1,invoice_text])
    mock_labels.append([1+1,name,f"{street},{city},{state},{zip}",f"INV-2023-{invnos}",f"{year}-{invmonth}-{invday}",f"{year}-{duemonth}-{dueday}",balance])
    i=i+1

# Print the first mock invoice and its labels for reference (will use the list indexes for id)
# print(mock[0])
# print(mock_labels[0])
print("Finished")
mockdf = pd.DataFrame(mock,columns=("id",'sentence'))
mocklabelsdf = pd.DataFrame(mock_labels,columns=('id',"name","address","invoicenumber","invoicedate","duedate","balance"))



Finished


In [3]:
mock

[[1,
  '\n    Dear Customer,\n\n    Invoice Number: INV-2023-222\n    Invoice Date: 2023-7-16\n    Due Date: 2023-8-20\n\n    Billing Information:\n    Recipient Name: Mark Brown\n    Recipient Address: 03884 Lindsey Villages, Hensonburgh, West Virginia 36638\n\n    Product Details:\n    - Description: oil Monitor K4\n      Quantity: 1\n      Unit Price: $203.39\n    - Description: same Headphones H6\n      Quantity: 5\n      Unit Price: $185.06\n\n    Balance Due: $639115.9648748989\n\n    Thank you for your business!\n    '],
 [2,
  '\n    Dear Customer,\n\n    Invoice Number: INV-2023-639\n    Invoice Date: 2023-1-17\n    Due Date: 2023-3-4\n\n    Billing Information:\n    Recipient Name: Terri Jackson\n    Recipient Address: 57349 Williams River, Nancyhaven, Tennessee 02776\n\n    Product Details:\n    - Description: from Keyboard M2\n      Quantity: 4\n      Unit Price: $964.38\n    - Description: leg Mouse H6\n      Quantity: 1\n      Unit Price: $150.45\n\n    Balance Due: $9450

## Running 1000 entries 

In [8]:
combined = []

i = 0 
while i < (len(mockdf)):
    # Process text using the loaded model
    tokenlist = []
    entitylist = []
    text = mockdf['sentence'][i]
    doc = nlp(text)
    # # Access token-level information
    # for token in doc:
    #     tokenlist.append([token.text, token.pos_])

    # Access named entities
    for ent in doc.ents:
        entitylist.append([ent.text, ent.label_,ent.end_char,ent.start_char,])
    combined.append([i+1,[entitylist]])
    i = i +1

In [5]:
results = pd.DataFrame(combined,columns=('id','results'))

In [6]:
results.to_json("./files/results.json")
results.to_csv("./files/results.csv")